In [1]:
# @title notebook setup
if 'colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/fnc_agrp_project/fnc_analysis_code')
from fnc_analysis import *
from scipy import stats as st
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

In [2]:
#@title load tidy data and subset dataframes for later use

#load unlimited intake
df=pd.read_csv('../pre_processed/unlimited_intake/total_licks_normal_intake.csv',index_col=[0,1,2]).sort_index()
dft=pd.read_csv('../pre_processed/unlimited_intake/lick_times_normal_intake.csv',index_col=[0,1,2]).sort_index()

#load limited intake
df_ltd=pd.read_csv('../pre_processed/limited_intake/total_licks_limited_intake.csv',index_col=[0,1,2]).sort_index()
dft_ltd=pd.read_csv('../pre_processed/limited_intake/lick_times_limited_intake.csv',index_col=[0,1,2]).sort_index()

#load sated
df_s=pd.read_csv('../pre_processed/sated/total_licks_sated.csv',index_col=[0,1,2]).sort_index()
dft_s=pd.read_csv('../pre_processed/sated/lick_times_sated.csv',index_col=[0,1,2]).sort_index()


#select glucose data
#unlimited intake
df_test_g=df.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'glucose',:,'test']
dft_test_g=dft.set_index([ 'phase','day','CS'],append=True).loc[:,'glucose',:,'test']

df_train_g = df.set_index([ 'phase', 'CS' ],append=True).loc[:,'glucose',:,'training']
dft_train_g = dft.set_index([ 'phase', 'CS' ],append=True).loc[:,'glucose',:,'training']

#limited intake
df_test_g_ltd = df_ltd.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'glucose',:,'test']
dft_test_g_ltd = dft_ltd.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'glucose',:,'test']

df_train_g_ltd = df_ltd.set_index([ 'phase', 'CS' ],append=True).loc[:,'glucose',:,'training']
dft_train_g_ltd = dft_ltd.set_index([ 'phase', 'CS' ],append=True).loc[:,'glucose',:,'training']

#sated
df_test_g_st= df_s.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'glucose',:,'test']
dft_test_g_st= dft_s.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'glucose',:,'test']

df_train_g_st = df_s.set_index([ 'phase', 'CS' ],append=True).loc[:,'glucose',:,'training']
dft_train_g_st = dft_s.set_index([ 'phase', 'CS' ],append=True).loc[:,'glucose',:,'training']

#select fat data
#unlimited intake
df_test_f=df.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'fat',:,'test']
dft_test_f=dft.set_index([ 'phase','day','CS'],append=True).loc[:,'fat',:,'test']

df_train_f = df.set_index([ 'phase', 'CS' ],append=True).loc[:,'fat',:,'training']
dft_train_f = dft.set_index([ 'phase', 'CS' ],append=True).loc[:,'fat',:,'training']

#limited intake
df_test_f_ltd = df_ltd.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'fat',:,'test']
dft_test_f_ltd = dft_ltd.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'fat',:,'test']

df_train_f_ltd = df_ltd.set_index([ 'phase', 'CS' ],append=True).loc[:,'fat',:,'training']
dft_train_f_ltd = dft_ltd.set_index([ 'phase', 'CS' ],append=True).loc[:,'fat',:,'training']

#sated
df_test_f_st= df_s.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'fat',:,'test']
dft_test_f_st= dft_s.set_index([ 'phase', 'day', 'CS' ], append=True).loc[:,'fat',:,'test']

df_train_f_st = df_s.set_index([ 'phase', 'CS' ],append=True).loc[:,'fat',:,'training']
dft_train_f_st = dft_s.set_index([ 'phase', 'CS' ],append=True).loc[:,'fat',:,'training']

In [3]:
# determine the amount of time post infusion before we need to take the controls bottles away
# we do this by determining a lick threshold based on the normal intake controls and figuring out the 
# average time it took the normal intake stim mice to reach that threshold

# get the average licks of the controls per session
thresh = df_train_g.loc[False,].pivot_table(index=['day','sess'], values = 'total_licks').round().total_licks
lick_times = dft_train_g.set_index(['day','sess','lick_number'], append=True).reset_index('CS').loc[True,].time

tmp = {}
for (day,sess), th in thresh.items():
    # for each day/sess get the time post-infusion when all stim mice reached the respective threshold
    t = (lick_times.loc[:, day, sess, th-1] - lick_times.loc[:, day, sess, 19])/60 - 5
    tmp.update( {(day, sess) : t } )
    
thresh_times = pd.concat(tmp, names = ['day','sess']).unstack(['day','sess'])

# make sure we include any mice that never 
mice = lick_times.index.get_level_values('mouse').unique()
mice = mice[~mice.isin(thresh_times.index)]
empty_arr = np.ones((len(mice), thresh_times.columns.size))*np.nan
empty_df = pd.DataFrame(empty_arr, index = mice, columns = thresh_times.columns)
thresh_times = pd.concat((thresh_times, empty_df), axis =0 )

# all the nan values represent sessions where a given mouse didn't reach the threshold
# here we fill those nan values with the time from the hypothetical end of the infusion (assuming 5 min inf for limited intake)
# to the end of the 30 min sess
t_end = lick_times.groupby(['mouse','day','sess']).apply(lambda x: (1800 - x.loc[:,:,:,19].values[0])/60 - 5).unstack(['day','sess'])
thresh_times.update(t_end, filter_func = pd.isnull)
thresh_times.mean(axis=0, skipna=True)


day  sess
0.0  0       7.000274
     1       8.701810
1.0  0       9.745643
     1       6.607607
2.0  0       6.400179
     1       5.458131
dtype: float64

In [4]:
thresh

day  sess
0.0  0       360.0
     1       905.0
1.0  0       986.0
     1       724.0
2.0  0       751.0
     1       921.0
Name: total_licks, dtype: float64

In [5]:
df_train_g_ltd.loc[False,].pivot_table(index=['day','sess'], values = 'total_licks').round()

total_licks
day sess             
0.0 0           374.0
    1           623.0
1.0 0           610.0
    1           531.0
2.0 0           507.0
    1           816.0

In [9]:
df_train_g_st.loc[False,].pivot_table(index=['day','sess'], values = 'total_licks').round()

total_licks
day sess             
0.0 0            49.0
    1            42.0
1.0 0            60.0
    1           107.0
2.0 0            78.0
    1            73.0

In [8]:
df_train_f_st.loc[False,].pivot_table(index=['day','sess'], values = 'total_licks').round()

total_licks
day sess             
0.0 0            66.0
    1            87.0
1.0 0            76.0
    1           148.0
2.0 0           141.0
    1           126.0